### 러닝머신 : 회귀
회귀분석: 최적의 회귀계수를 찾아내는 것 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

<선형회귀> 회귀계수가 선형
일반선형회귀: 단일회귀/ 예측값과 실제값의 오류를 최소화할 수 있도록 회귀계수를 최적화/규제 없음<br>
            from sklearn.linear_model import LinearRegression
다항회귀:  전처리 클래스(변수의 변경이므로) <br>
릿지: 선형회귀 + L2, 규제 강할 수록 회귀계수가 작아짐(예측성 떨어짐)<br>
라쏘: 선형회귀 + L1<br>
엘레스틱넷: L2, L1규제 결합<br>
SGD: 확률적인 경사하강법 적용/ 가중치 업데이트하면 찾음

<회귀의 성능평가>  숫자 클수록 안좋음  <BR>
MAE: 실제값과 예측값 차이의 절대값들의 평균  neg_mean_absolute_error
MSE: 실제값과 예측값 차이의 제곱, 평균   neg_mean_squared_error
RMSE: MSE의 루트값                  neg_root_mean_squared_error
R2: 분석기반 예측성능 평가            r2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
# 예측성능 측정/평가 함수 : mean_squared_error, mean_absolute_error, r2_score, 
# root_mean_squared_error, root_mean_square_log_error

# 1. RMSLE(root_mean_squar_log_error) 함수
def rmsle (y, pred):
    log_y=np.log1p(y)
    log_pred=np.log1p(pred)
    squared_error=(log_y-log_pred)**2
    rmsle=np.sqrt(np.mean(squared_error))
    return rmsle
# 2. RMSE(root_mean_squared_error) 함수
def rmse(y, pred): 
    return np.sqrt(mean_squared_error(y, pred))

# 3. 모든 예측성능 측정 함수
def evaluate_regr(y,pred):
    rmsle_val=rmsle(y,pred)
    rmse_val=rmse(y,pred)
    mae_val=mean_absolute_error(y,pred)
    r2_val=r2_score(y,pred)
    print('RMSLE: {0:.2f}, RMSE: {1:.2f}, MAE: {2:.2f}, R2: {3:.2f}'.format(rmsle_val,rmse_val, mae_val, r2_val)) 

In [ ]:
# 2. 실제값과 예측값의 차이 확인
def get_top_error_data(y_test, pred, n_top=5):
    result_df=pd.DataFrame(y_test.values, columns=['real_count'])
    result_df['pred_count']=np.round(pred)
    result_df['diff']=np.abs(result_df['real_count'] - result_df['pred_count'])
    print(result_df.sort_values('diff', ascending=False)[:n_top])

In [ ]:
# 3.개별 featuer의 회귀계수값 시각화
coef=pd.Series(lr_reg.coef_, index=X_features.columns)
coef_sort=coef.sort_values(ascending=False)
sns.barplot(x=coef_sort.values, y=coef_sort.index)

In [ ]:
# 4. 모델과 학습/테스트 데이터세트를 입력하면 성능평가 수치 반환
def get_model_predict(model, X_train, X_test,  y_train, y_test, is_expm1=False):
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    if is_expm1:
        y_test=np.expm1(y_test)
        pred=np.expm1(pred)
    print(model.__class__.__name__)
    evaluate_regr(y_test, pred)

In [ ]:
# 5. 회귀계수 상위 20 그래프
coef=pd.Series(lr_reg.coef_, index=X_features_ohe.columns)
coef_sort=coef.sort_values(ascending=False)[:20]
sns.barplot(x=coef_sort.values, y=coef_sort.index)

In [ ]:
alphas=[0,0.1,1,10,100]
for alpha in alphas:
    ridge=Ridge(alpha=alpha)
    neg_mse_scores=cross_val_score(ridge,X_features, y_target, scoring='neg_mean_squared_error',cv=5)
    avg_rmse= np.mean(np.sqrt(-1*neg_mse_scores))
    print(f'alpha {alpha} 일 때 5fold의 평균 RMSE: {avg_rmse:.3f}')

In [ ]:
# alpha값에 따는 회귀모델의 폴드평균 RMSE를 출력/회귀계수들을 Dataframe으로 반환
def get_linear_reg_eval(model_name, params=None, X_data=None, y_target=None, 
                        verbose=True, return_coeff=True):
    coeff_df=pd.DataFrame()
    if verbose : print(model_name)
    for param in params:
        if model_name == 'Ridge': model = Ridge(alpha=param)
        elif model_name == 'Lasso' : model = Lasso(alpha=param)
        elif model_name == 'ElasticNet' : model = ElasticNet(alpha=param, l1_ratio=0.7)
        neg_mse_scores = cross_val_score(model, X_data, y_target, 
                                         scoring='neg_mean_squared_error', cv=5)
        avg_rmse=np.mean(np.sqrt(-1*neg_mse_scores))
        print('alpha {0}일때 5폴드 평균 RMSE: {1:.3f}'.format(param, avg_rmse))
        
        model.fit(X_data,y_target)
        if return_coeff:
            coeff=pd.Series(data=model.coef_, index=X_data.columns)
            colname='alpha:'+str(param)
            coeff_df[colname]=coeff
    return coeff_df
# 적용
alphas=[0.07,0.1,0.5,1,3]
coeff_lasso_df=get_linear_reg_eval('Lasso',params=alphas, X_data=X_features, y_target=y_target)

In [ ]:
# 선형회귀모델의 스케일링,정규화 함수-중요 피처나 타겟값이 왜곡되었을때
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures

def get_scaled_data(method=None,p_degree=None, input_data=None):
    if method =='Standard':
        scaled_data=StandardScaler().fit_transform(input_data)
    elif method =='MinMax':
        scaled_data=MinMaxScaler().fit_transform(input_data)
    elif method =='Log':
        scaled_data=np.log1p(input_data)
    else: scaled_data=input_data
    
    if p_degree !=None:
        scaled_data=PolynomialFeatures(degree=p_degree, 
                                       include_bias=False).fit_transform(scaled_data)
    return scaled_data
#적용
X_data_scaled=get_scaled_data('Log',input_data=X_features)
alphas=[0.1,1,10,100]
get_linear_reg_eval('Ridge',params=alphas, X_data=X_data_scaled,y_target=y_target, verbose=False, return_coeff=False)

#### 회귀의 종류

In [2]:
# 일반 선형회귀/ 단순선형회귀/단일회귀
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import load_diabetes

diabetes= load_diabetes()
y= diabetes.target
X= diabetes.data
lr=LinearRegression()
neg_mse_scores=cross_val_score(lr,X,y, scoring='neg_mean_squared_error',cv=5)
rmse_scores=np.sqrt(-1*neg_mse_scores)
avg_rmse= np.mean(rmse_scores)

print('5fold 개별 Negative MSE scores: ', np.round(neg_mse_scores, 2))
print('5fold 개별 RMSE scores: ', np.round(rmse_scores, 2))
print(f'5fold 평균 RMSE: {avg_rmse:3f}')

5fold 개별 Negative MSE scores:  [-2779.92 -3028.84 -3237.69 -3008.75 -2910.21]
5fold 개별 RMSE scores:  [52.72 55.03 56.9  54.85 53.95]
5fold 평균 RMSE: 54.691791


In [ ]:
# 다항회귀 : 단항식피처를 degree에 해당하는 다항식피처로 변환
from sklearn.preprocessing import PolynomialFeatures

poly_ftr=PolynomialFeatures(degree=2).fit_transform(X)
model=LinearRegression()
model.fit(poly_ftr, y)
print('Polynomial 회귀계수: \n', np.round(model.coef_,2))
print('Polynomial 회귀 shape: \n', model.coef_,shape)

In [8]:
# 릿지 회귀: L2규제, alpha값은 Lasso보다 큰 수로
from sklearn.linear_model import Ridge
ridge=Ridge(alpha=10)
neg_mse_scores=cross_val_score(ridge,X,y, scoring='neg_mean_squared_error',cv=5)
rmse_scores=np.sqrt(-1*neg_mse_scores)
avg_rmse= np.mean(rmse_scores)

print('5fold 개별 Negative MSE scores: ', np.round(neg_mse_scores, 2))
print('5fold 개별 RMSE scores: ', np.round(rmse_scores, 2))
print(f'5fold 평균 RMSE: {avg_rmse:3f}')

5fold 개별 Negative MSE scores:  [-4540.48 -5430.5  -5284.4  -4364.15 -5463.35]
5fold 개별 RMSE scores:  [67.38 73.69 72.69 66.06 73.91]
5fold 평균 RMSE: 70.749030


In [11]:
# 적절한 릿지 alpha(규제) 값 찾기
alphas=[0,0.1,1,10,100]
for alpha in alphas:
    ridge=Ridge(alpha=alpha)
    neg_mse_scores=cross_val_score(ridge,X,y, scoring='neg_mean_squared_error',cv=5)
    avg_rmse= np.mean(np.sqrt(-1*neg_mse_scores))
    print(f'alpha {alpha} 일 때 5fold의 평균 RMSE: {avg_rmse:.3f}')

alpha 0 일 때 5fold의 평균 RMSE: 54.692
alpha 0.1 일 때 5fold의 평균 RMSE: 54.825
alpha 1 일 때 5fold의 평균 RMSE: 58.449
alpha 10 일 때 5fold의 평균 RMSE: 70.749
alpha 100 일 때 5fold의 평균 RMSE: 76.399


In [14]:
#라쏘 회귀: L1규제 alpha갑 0~1 사이로
from sklearn.linear_model import Lasso
alphas=[0, 0.01, 0.1, 1, 10]
for alpha in alphas:
    lasso=Lasso(alpha=alpha)
    neg_mse_scores=cross_val_score(lasso,X,y, scoring='neg_mean_squared_error',cv=5)
    avg_rmse= np.mean(np.sqrt(-1*neg_mse_scores))
    print(f'alpha {alpha} 일 때 5fold의 평균 RMSE: {avg_rmse:.3f}')

alpha 0 일 때 5fold의 평균 RMSE: 54.692
alpha 0.01 일 때 5fold의 평균 RMSE: 54.756
alpha 0.1 일 때 5fold의 평균 RMSE: 54.843
alpha 1 일 때 5fold의 평균 RMSE: 62.009
alpha 10 일 때 5fold의 평균 RMSE: 77.264


In [15]:
# 엘라스틱넷 회귀: L1, L2 규제를 비율로 정함/ alpha, l1_ratio 파라미터 -잘 사용 안함
from sklearn.linear_model import ElasticNet
alphas=[0, 0.01, 0.1, 1, 10]
for alpha in alphas:
    elastic=ElasticNet(alpha=alpha)
    neg_mse_scores=cross_val_score(elastic,X,y, scoring='neg_mean_squared_error',cv=5)
    avg_rmse= np.mean(np.sqrt(-1*neg_mse_scores))
    print(f'alpha {alpha} 일 때 5fold의 평균 RMSE: {avg_rmse:.3f}')

alpha 0 일 때 5fold의 평균 RMSE: 54.692
alpha 0.01 일 때 5fold의 평균 RMSE: 61.112
alpha 0.1 일 때 5fold의 평균 RMSE: 73.203
alpha 1 일 때 5fold의 평균 RMSE: 76.925
alpha 10 일 때 5fold의 평균 RMSE: 77.264


In [16]:
# SGD 확율적 경사하강법을 적용 :부스팅방식(데이터를 하나씩 학습하며 가중치 업데이트)
from sklearn.linear_model import SGDRegressor
alphas=[0, 0.01, 0.1, 1, 10]
for alpha in alphas:
    sgd_reg=SGDRegressor(alpha=alpha)
    neg_mse_scores=cross_val_score(sgd_reg,X,y, scoring='neg_mean_squared_error',cv=5)
    avg_rmse= np.mean(np.sqrt(-1*neg_mse_scores))
    print(f'alpha {alpha} 일 때 5fold의 평균 RMSE: {avg_rmse:.3f}')

alpha 0 일 때 5fold의 평균 RMSE: 59.021
alpha 0.01 일 때 5fold의 평균 RMSE: 65.430
alpha 0.1 일 때 5fold의 평균 RMSE: 74.983
alpha 1 일 때 5fold의 평균 RMSE: 76.998
alpha 10 일 때 5fold의 평균 RMSE: 77.252


In [18]:
# 비선형 회귀:회귀 트리-선형으로 표현되지 않는 모든/ node를 생성하는 방식이 다름
# 일반회귀기법 보다 예측 성능 높음
from sklearn.ensemble import RandomForestRegressor

rf_reg=RandomForestRegressor(random_state=0, n_estimators=100)
neg_mse_scores=cross_val_score(rf_reg,X,y, scoring='neg_mean_squared_error',cv=5)
rmse_scores=np.sqrt(-1*neg_mse_scores)
avg_rmse= np.mean(rmse_scores)

print('5fold 개별 Negative MSE scores: ', np.round(neg_mse_scores, 2))
print('5fold 개별 RMSE scores: ', np.round(rmse_scores, 2))
print(f'5fold 평균 RMSE: {avg_rmse:3f}')

5fold 개별 Negative MSE scores:  [-3028.85 -3104.98 -3547.87 -3392.66 -3616.43]
5fold 개별 RMSE scores:  [55.04 55.72 59.56 58.25 60.14]
5fold 평균 RMSE: 57.740947


In [ ]:
# 회귀트리-트리그림 그리기
from sklearn.tree import plot_tree
plt.figure(dpi = 1200)
plot_tree(dt_reg)
plt.savefig('a.png')

In [ ]:
# 중요 peature 그리기
rf_reg.fit(X, y)
ftr_importances_values = rf_reg.feature_importances_
ftr_importances = pd.Series(ftr_importances_values,index=X.columns)
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]
plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20 , y = ftr_top20.index)
plt.show()

In [19]:
# 다른 비선형회귀 모델들과 비교
def get_model_cv_prediction(model, X, y):
    neg_mse_scores = cross_val_score(model, X, y,
    scoring="neg_mean_squared_error", cv = 5)
    rmse_scores = np.sqrt(-1 * neg_mse_scores)
    avg_rmse = np.mean(rmse_scores)
    print(' ',model.__class__.__name__ , ' ')
    print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))

In [20]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
dt_reg = DecisionTreeRegressor(random_state=0, max_depth=4)
gb_reg = GradientBoostingRegressor(random_state=0, n_estimators=1000)
xgb_reg = XGBRegressor(random_state=0, n_estimators=1000)
models = [dt_reg, rf_reg, gb_reg, xgb_reg]
for model in models: 
    get_model_cv_prediction(model, X, y)

  DecisionTreeRegressor  
 5 교차 검증의 평균 RMSE : 64.320 
  RandomForestRegressor  
 5 교차 검증의 평균 RMSE : 57.741 
  GradientBoostingRegressor  
 5 교차 검증의 평균 RMSE : 65.495 
  XGBRegressor  
 5 교차 검증의 평균 RMSE : 63.035 
